<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 824ms/step - loss: 0.6706 - accuracy: 0.5766 - val_loss: 0.6376 - val_accuracy: 0.8244
Epoch 2/10
26/26 [==============================] - 20s 776ms/step - loss: 0.6089 - accuracy: 0.7105 - val_loss: 0.5707 - val_accuracy: 0.7805
Epoch 3/10
26/26 [==============================] - 20s 766ms/step - loss: 0.5589 - accuracy: 0.7336 - val_loss: 0.5573 - val_accuracy: 0.7171
Epoch 4/10
26/26 [==============================] - 20s 769ms/step - loss: 0.5100 - accuracy: 0.7762 - val_loss: 0.7429 - val_accuracy: 0.5073
Epoch 5/10
26/26 [==============================] - 20s 765ms/step - loss: 0.4226 - accuracy: 0.8163 - val_loss: 0.3447 - val_accuracy: 0.9024
Epoch 6/10
26/26 [==============================] - 20s 773ms/step - loss: 0.3875 - accuracy: 0.8528 - val_loss: 0.3025 - val_accuracy: 0.9171
Epoch 7/10
26/26 [==============================] - 20s 766ms/step - loss: 0.3021 - accuracy: 0.8917 - val_loss: 0.2546 - val_accuracy: 0.9220

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.63.h5
26/26 - 21s - loss: 0.6652 - accuracy: 0.6131 - val_loss: 0.6295 - val_accuracy: 0.6585
Epoch 2/5

Epoch 00002: saving model to weights.02-0.56.h5
26/26 - 20s - loss: 0.5933 - accuracy: 0.7311 - val_loss: 0.5551 - val_accuracy: 0.7854
Epoch 3/5

Epoch 00003: saving model to weights.03-0.51.h5
26/26 - 20s - loss: 0.5218 - accuracy: 0.7482 - val_loss: 0.5124 - val_accuracy: 0.7951
Epoch 4/5

Epoch 00004: saving model to weights.04-0.46.h5
26/26 - 20s - loss: 0.4394 - accuracy: 0.8102 - val_loss: 0.4590 - val_accuracy: 0.8146
Epoch 5/5

Epoch 00005: saving model to weights.05-0.36.h5
26/26 - 21s - loss: 0.3895 - accuracy: 0.8248 - val_loss: 0.3618 - val_accuracy: 0.8780


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6766 - accuracy: 0.5742 - val_loss: 0.6783 - val_accuracy: 0.4390


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6646 - accuracy: 0.5852 - val_loss: 0.6684 - val_accuracy: 0.4634
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 21s - loss: 0.6155 - accuracy: 0.7044 - val_loss: 0.7034 - val_accuracy: 0.4488


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6599 - accuracy: 0.5839 - val_loss: 0.6928 - val_accuracy: 0.4585
Epoch 2/50
26/26 - 20s - loss: 0.6074 - accuracy: 0.6861 - val_loss: 0.5838 - val_accuracy: 0.7268
Epoch 3/50
26/26 - 20s - loss: 0.5252 - accuracy: 0.7676 - val_loss: 0.4764 - val_accuracy: 0.8195
Epoch 4/50
26/26 - 20s - loss: 0.4598 - accuracy: 0.8005 - val_loss: 0.4080 - val_accuracy: 0.8341
Epoch 5/50
26/26 - 20s - loss: 0.4014 - accuracy: 0.8443 - val_loss: 0.3549 - val_accuracy: 0.8780
Epoch 6/50
26/26 - 20s - loss: 0.3216 - accuracy: 0.8942 - val_loss: 0.2832 - val_accuracy: 0.9122
Epoch 7/50
26/26 - 21s - loss: 0.2804 - accuracy: 0.9015 - val_loss: 0.2090 - val_accuracy: 0.9415
Epoch 8/50
26/26 - 20s - loss: 0.2356 - accuracy: 0.9270 - val_loss: 0.1649 - val_accuracy: 0.9610
Epoch 9/50
26/26 - 20s - loss: 0.1695 - accuracy: 0.9623 - val_loss: 0.2457 - val_accuracy: 0.9073
Epoch 10/50
26/26 - 20s - loss: 0.1374 - accuracy: 0.9647 - val_loss: 0.1205 - val_accuracy: 0.9610
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 780ms/step - loss: 0.6767 - accuracy: 0.5596 - val_loss: 0.6595 - val_accuracy: 0.7756
Epoch 2/5
26/26 [==============================] - 20s 783ms/step - loss: 0.6352 - accuracy: 0.6873 - val_loss: 0.6119 - val_accuracy: 0.7854
Epoch 3/5
26/26 [==============================] - 20s 766ms/step - loss: 0.5808 - accuracy: 0.7287 - val_loss: 0.5418 - val_accuracy: 0.8049
Epoch 4/5
26/26 [==============================] - 20s 784ms/step - loss: 0.5051 - accuracy: 0.7871 - val_loss: 0.4646 - val_accuracy: 0.7659
Epoch 5/5
26/26 [==============================] - 20s 777ms/step - loss: 0.4504 - accuracy: 0.8114 - val_loss: 0.4951 - val_accuracy: 0.7512


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.559611,0.676576,0.775610,0.659539
1,1,0.687348,0.635440,0.785366,0.611872
2,2,0.728710,0.581140,0.804878,0.541821
3,3,0.787105,0.504987,0.765854,0.464581
4,4,0.811435,0.450029,0.751219,0.495124


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 33s 1s/step - loss: 0.6672 - accuracy: 0.6010 - val_loss: 0.6457 - val_accuracy: 0.7122

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 803ms/step - loss: 0.6308 - accuracy: 0.7080 - val_loss: 0.6380 - val_accuracy: 0.6780

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 21s 796ms/step - loss: 0.6135 - accuracy: 0.7226 - val_loss: 0.6256 - val_accuracy: 0.7415

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 777ms/step - loss: 0.6048 - accuracy: 0.7603 - val_loss: 0.6244 - val_accuracy: 0.7171

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 21s 800ms/step - loss: 0.5994

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 23s 896ms/step - loss: 0.6802 - accuracy: 0.5365 - val_loss: 0.6762 - val_accuracy: 0.5268
Epoch 2/50
26/26 [==============================] - 21s 811ms/step - loss: 0.6506 - accuracy: 0.6253 - val_loss: 0.6364 - val_accuracy: 0.8732
Epoch 3/50
26/26 [==============================] - 20s 781ms/step - loss: 0.6079 - accuracy: 0.7518 - val_loss: 0.5992 - val_accuracy: 0.6049
Epoch 4/50
26/26 [==============================] - 21s 804ms/step - loss: 0.5420 - accuracy: 0.8212 - val_loss: 0.4943 - val_accuracy: 0.8976
Epoch 5/50
12/26 [============>.................] - ETA: 11s - loss: 0.5023 - accuracy: 0.8255

In [ ]:
%tensorboard --logdir log_dir